In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np   
import pandas as pd  

import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
RAW_DATA_PATH_X = '/Users/piyush/Desktop/dsml_Portfolio/visa/data/processed/X.csv'
RAW_DATA_PATH_y = '/Users/piyush/Desktop/dsml_Portfolio/visa/data/processed/y.csv'
X = pd.read_csv(RAW_DATA_PATH_X)
y = pd.read_csv(RAW_DATA_PATH_y)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1, stratify=y)

In [4]:
xgb_tuned = XGBClassifier(random_state=1, eval_metric='logloss')

# Grid of parameters to choose from
parameters = {
    "n_estimators": [10,30,50],
    "scale_pos_weight":[1,2,5],
    "subsample":[0.7,0.9,1],
    "learning_rate":[0.05, 0.1,0.2],
    "colsample_bytree":[0.7,0.9,1],
    "colsample_bylevel":[0.5,0.7,1]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# Run the grid search
grid_obj = GridSearchCV(xgb_tuned, parameters,scoring=scorer,cv=3)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
xgb_tuned = grid_obj.best_estimator_

# Fit the best algorithm to the data.
xgb_tuned.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=0.5, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.2, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=10, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=1, ...)

In [5]:
pred = xgb_tuned.predict(X_test)

acc = accuracy_score(y_test, pred)  # to compute Accuracy
recall = recall_score(y_test, pred)  # to compute Recall
precision = precision_score(y_test, pred)  # to compute Precision
f1 = f1_score(y_test, pred)  # to compute F1-score

    # creating a dataframe of metrics
df_perf = pd.DataFrame(
        {
            "Accuracy": acc,
            "Recall": recall,
            "Precision": precision,
            "F1": f1,
        },
        index=[0],
    )

In [6]:
df_perf

,Accuracy,Recall,Precision,F1
0,0.746599,0.871107,0.776633,0.821161


In [7]:
pred = xgb_tuned.predict(X_train)

acc = accuracy_score(y_train, pred)  # to compute Accuracy
recall = recall_score(y_train, pred)  # to compute Recall
precision = precision_score(y_train, pred)  # to compute Precision
f1 = f1_score(y_train, pred)  # to compute F1-score

    # creating a dataframe of metrics
df_perf = pd.DataFrame(
        {
            "Accuracy": acc,
            "Recall": recall,
            "Precision": precision,
            "F1": f1,
        },
        index=[0],
    )

In [8]:
df_perf

,Accuracy,Recall,Precision,F1
0,0.757401,0.874003,0.786524,0.827959
